# Titanic - Machine Learning from Disaster

## Imports

In [2]:
import opendatasets as od

In [3]:
import pandas as pd

In [4]:
import numpy as np

## 1. Getting the Data

In [5]:
!pip install opendatasets

In [6]:
od.download("https://www.kaggle.com/competitions/titanic/data")


Skipping, found downloaded files in ".\titanic" (use force=True to force download)


In [7]:
train_set = pd.read_csv('titanic/train.csv')

In [8]:
test_set = pd.read_csv('titanic/test.csv')

In [9]:
survived = pd.read_csv('titanic/gender_submission.csv')

In [10]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
survived.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## 2. Studying the Data

In [12]:
np.where(pd.isnull(train_set['Cabin']))

(array([  0,   2,   4,   5,   7,   8,   9,  12,  13,  14,  15,  16,  17,
         18,  19,  20,  22,  24,  25,  26,  28,  29,  30,  32,  33,  34,
         35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
         48,  49,  50,  51,  53,  56,  57,  58,  59,  60,  63,  64,  65,
         67,  68,  69,  70,  71,  72,  73,  74,  76,  77,  78,  79,  80,
         81,  82,  83,  84,  85,  86,  87,  89,  90,  91,  93,  94,  95,
         98,  99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 111, 112,
        113, 114, 115, 116, 117, 119, 120, 121, 122, 125, 126, 127, 129,
        130, 131, 132, 133, 134, 135, 138, 140, 141, 142, 143, 144, 145,
        146, 147, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160,
        161, 162, 163, 164, 165, 167, 168, 169, 171, 172, 173, 175, 176,
        178, 179, 180, 181, 182, 184, 186, 187, 188, 189, 190, 191, 192,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 210,
        211, 212, 213, 214, 216, 217, 219, 220, 221